In [11]:
import os
import tensorflow as tf
import pickle
from utils import DataLoader
import re
import numpy as np
working_dir = os.getcwd()
data_dir = os.path.join(working_dir, "data")
session = tf.InteractiveSession()

In [25]:
x = tf.constant(np.random.random((20, 100)), dtype=tf.float32)

In [29]:
ta = tf.TensorArray(tf.float32, 2)

In [30]:
ta = ta.write(0, x)

In [31]:
ta.stack().eval()

InvalidArgumentError: TensorArray TensorArray_6_5: Could not read from TensorArray index 1 because it has not yet been written to.
	 [[Node: TensorArrayStack_6/TensorArrayGatherV3 = TensorArrayGatherV3[_class=["loc:@TensorArray_6"], dtype=DT_FLOAT, element_shape=[20,100], _device="/job:localhost/replica:0/task:0/cpu:0"](TensorArray_6, TensorArrayStack_6/range, TensorArrayWrite_6/TensorArrayWriteV3)]]

Caused by op 'TensorArrayStack_6/TensorArrayGatherV3', defined at:
  File "c:\users\administrator\appdata\local\programs\python\python35\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\administrator\appdata\local\programs\python\python35\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\administrator\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\users\administrator\appdata\local\programs\python\python35\lib\site-package

In [34]:
X = tf.constant(np.random.random((20, 100, 200)))

In [35]:
W = tf.constant(np.random.random((200, 200)))

In [38]:
tf.matmul(X, tf.reshape(W, [1, 200, 200]))

ValueError: Dimension 0 in both shapes must be equal, but are 20 and 1 for 'MatMul_2' (op: 'BatchMatMul') with input shapes: [20,100,200], [1,200,200].

In [22]:
time = tf.constant(0, dtype=tf.int32)
time += 1

In [23]:
o = tf.constant(0, dtype=tf.int32)
oo  = tf.constant(0, dtype=tf.int32)

In [30]:
tf.stack([o, time, o], axis=0)

<tf.Tensor 'stack_4:0' shape=(3,) dtype=int32>

In [20]:
tf.constant[3]

TypeError: 'function' object is not subscriptable

In [2]:
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file

In [5]:
print_tensors_in_checkpoint_file(file_name='./save_pointer_run/model.ckpt-49500', tensor_name='', all_tensors=True)

tensor_name:  optimizer/beta2_power
3.09797e-22
tensor_name:  quesBiRNN/cell_0/bidirectional_rnn/fw/gru_cell/candidate/bias/Adam
[ -2.99454996e-06  -5.93463010e-05   4.99954012e-06  -3.21467178e-06
  -1.60719515e-04  -6.83834241e-06  -1.97801364e-06   5.71531609e-06
  -1.41668797e-05   5.00053866e-05  -7.08240850e-06  -1.82305311e-03
   8.94877940e-06  -5.28462351e-06  -1.57198592e-05  -9.07131744e-06
  -5.31043224e-06   5.00849215e-04   5.88952275e-07   4.54738365e-05
  -8.58547537e-06   5.23210610e-06  -1.02404942e-06   4.86502415e-07
  -4.41469865e-07  -1.05210438e-05   1.46178161e-06  -1.30951768e-04
   5.01208124e-05   2.39517776e-05   9.19075392e-05   7.53929407e-06
   1.38567571e-04  -4.47400862e-06   5.61865396e-04   4.67654355e-02
   2.40246754e-05  -9.02484244e-05   7.95303276e-06   2.07034391e-04
   3.25061183e-06  -1.13221994e-07   1.29619579e-07  -1.20936093e-05
  -7.75899935e-06   2.17628281e-06   3.61491948e-05   6.01706051e-05
  -5.65375376e-05  -3.43593383e-05   5.8336

    5.33012379e-28   4.54775616e-28]]
tensor_name:  paraBiRNN/cell_0/bidirectional_rnn/fw/gru_cell/gates/bias/Adam
[ -4.65875352e-03  -5.52524107e-05  -8.73566081e-04   1.27175031e-03
  -8.85689929e-07  -1.53092190e-03   3.26192117e-06  -3.52075294e-05
  -3.45448861e-05   8.03203591e-07  -6.07448304e-03  -7.28255918e-06
  -1.16700260e-03   7.43856026e-06   3.28552369e-06  -7.60859519e-04
  -1.10888774e-04  -2.78063788e-04  -4.42066412e-05  -9.67905944e-05
   1.98799398e-06   2.65631388e-04   1.26254352e-04  -6.09546714e-06
  -4.60169977e-04  -3.52500996e-04  -2.74999843e-06  -1.81263764e-04
  -8.87149290e-05   5.66491690e-05   5.63446316e-04  -3.49005750e-05
   2.68569944e-04   3.40719562e-05  -8.79881263e-04  -1.14066454e-04
   1.97933055e-03  -9.07625690e-07  -8.79155152e-07  -2.00022769e-04
  -3.76006734e-04   4.30149294e-06   2.31700469e-04  -5.47723204e-04
   2.83083034e-04   8.98365033e-06   7.43209443e-04   1.64599231e-04
   1.54331894e-04   2.08609574e-03  -8.69098294e-05  -2.8

In [2]:
data_loader = DataLoader(data_dir=data_dir, embedding_dim=100, batch_size=20, training=True)

loading vocab files...
loading preprocessed data...


In [3]:
data = data_loader.load_json_data(os.path.join(data_loader.squad_dir, "train-v1.1.json"))["data"]

In [4]:
para_dict, para_to_qa_dict, qa_data_dict, discarded_dict = data_loader.parse_json_data(data)

In [76]:
really_discarded_dict = dict()
for d in discarded_dict:
    qaID = d
    paraID, q_words, question, a_words, answer = discarded_dict[qaID]
    para_words = para_dict[paraID]
    print("---")
    try:
        i, j = find_sub_list(a_words, para_words)
        print(para_words[i:j+1])
        print(a_words)
    except ValueError:
        print(qaID)
        really_discarded_dict[qaID] = paraID, q_words, question, a_words, answer
    print("---")

---
['start', 'value', 'of', 'a', 'routine', 'is', 'based', 'on', 'the', 'difficulty', 'of', 'the', 'elements', 'the', 'gymnast', 'attempts', 'and', 'whether', 'or', 'not', 'the', 'gymnast', 'meets', 'composition', 'requirements']
['e', 'start', 'value', 'of', 'a', 'routine', 'is', 'based', 'on', 'the', 'difficulty', 'of', 'the', 'elements', 'the', 'gymnast', 'attempts', 'and', 'whether', 'or', 'not', 'the', 'gymnast', 'meets', 'composition', 'requirements']
---
---
['.', 'the', 'resulting', 'indo', '-', 'scythian', 'kingdom', 'seems', 'to', 'have', 'gradually', 'pushed', 'the', 'remaining', 'indo', '-', 'greek']
['dom', '.', 'the', 'resulting', 'indo', '-', 'scythian', 'kingdom', 'seems', 'to', 'have', 'gradually', 'pushed', 'the', 'remaining', 'indo', '-', 'greek']
---
---
['zealander']
['new', 'zealand']
---
---
['known']
['kno']
---
---
['millionth']
['hundred', 'million']
---
---
['highest']
['228']
---
---
['government']
['28', 'july', '180']
---
---
['went']
['25']
---
---
['bil

---
---
['downplayed']
['downplay']
---
---
['maintained']
['generally', 'maintain']
---
---
['soviet']
['6']
---
---
['two']
['2']
---
---
['seven']
['7']
---
---
['sixth']
['six']
---
---
['southwestern']
['southwest']
---
---
['three']
['3']
---
---
['on']
['one']
---
---
['european']
['europe']
---
---
['two']
['2']
---
---
['two']
['2']
---
---
['not']
['no']
---
---
['three']
['3']
---
---
['is']
['i']
---
---
['school']
['privately', 'funded', 'english', 'language', 'schoo']
---
---
['one']
['1']
---
---
['three']
['3']
---
---
['western']
['west']
---
---
['fifteenth']
['15']
---
---
['businesses']
['the', 'company', "'", 's', 'mainstay', 'business']
---
---
['indirectly']
['indirect']
---
---
['in']
['9']
---
---
['had', 'become', '"', 'a', 'derelict', 'open', 'to', 'the', 'occupancy', 'of', 'every', 'enemy', ',', 'civilized', 'or', 'savage']
['orida', 'had', 'become', '"', 'a', 'derelict', 'open', 'to', 'the', 'occupancy', 'of', 'every', 'enemy', ',', 'civilized', 'or', 'sava

In [77]:
really_discarded_dict

{}

In [58]:
find_sub_list(["hey", "yolo"], ["1", "hey", "yolo", "3"])

(1, 2)

In [71]:
import re
import difflib
import inflect
p = inflect.engine()

def find_exact_sub_list(sublst, lst):
    """Given a list and a sublist, find the starting and ending indices of the sublist in the list."""
    for index in (i for i, e in enumerate(lst) if e == sublst[0]): #if list_word == sublist[0]
        if lst[index:index+len(sublst)] == sublst:
            return index, index + len(sublst) - 1
    raise ValueError("no exact match found between sublist and list!")
    
def find_sub_list(sublst, lst):
    try:
        return find_exact_sub_list(sublst, lst)
    except:
        pass
    if len(sublst) > 1:
        try:
            return find_sub_list(sublst[1:], lst)
        except:
            pass
        try:
            return find_sub_list(sublst[1:], lst)
        except:
            pass
    elif len(sublst) == 1:
        word = sublst[0]
        if bool(re.search(r'\d', word)): #if word contains a number
            try:
                number_word = p.number_to_words(word)
                return find_sub_list([number_word], lst)
            except:
                pass
        try: #find closest word in paragraph and choose that
            idx = np.argmax([difflib.SequenceMatcher(None, word, x).ratio() for x in lst])
            return (idx, idx)
            #similarities = [difflib.SequenceMatcher(None, word, x).ratio() for x in lst]
            #if max(similarities) > 0.4:
            #    idx = np.argmax(similarities)
            #    return (idx, idx)
        except:
            pass
    raise ValueError("approximate match between sublist and list not found!")

In [ ]:
sublst = ['t', 'parks', ',', 'schools', ',', 'public', 'buildings', ',', 'proper', 'roads', 'and', 'the', 'other', 'amenities', 'that', 'characterise', 'a', 'modern', 'city']

In [ ]:
lst = ['the', 'so', '-', 'called', '"', 'sack', 'of', 'palermo', '"', 'is', 'one', 'of', 'the', 'major', 'visible', 'faces', 'of', 'the', 'problem', '.', 'the', 'term', 'is', 'used', 'to', 'indicate', 'the', 'speculative', 'building', 'practices', 'that', 'have', 'filled', 'the', 'city', 'with', 'poor', 'buildings', '.', 'the', 'reduced', 'importance', 'of', 'agriculture', 'in', 'the', 'sicilian', 'economy', 'has', 'led', 'to', 'a', 'massive', 'migration', 'to', 'the', 'cities', ',', 'especially', 'palermo', ',', 'which', 'swelled', 'in', 'size', ',', 'leading', 'to', 'rapid', 'expansion', 'towards', 'the', 'north', '.', 'the', 'regulatory', 'plans', 'for', 'expansion', 'was', 'largely', 'ignored', 'in', 'the', 'boom', '.', 'new', 'parts', 'of', 'town', 'appeared', 'almost', 'out', 'of', 'nowhere', ',', 'but', 'without', 'parks', ',', 'schools', ',', 'public', 'buildings', ',', 'proper', 'roads', 'and', 'the', 'other', 'amenities', 'that', 'characterise', 'a', 'modern', 'city', '.']

In [ ]:
find_approx_sub_list(sublst, lst)

In [ ]:
import inflect
p = inflect.engine()

In [ ]:
float("3")

In [ ]:
p.number_to_words(3)

In [ ]:
max_seq_length = 100
batch_size = 10
rnn_size = 50
W_inputs = tf.constant(np.random.random((rnn_size, rnn_size)), dtype="float32")
W_pointerRNN = tf.constant(np.random.random((rnn_size, rnn_size)), dtype="float32")
v = tf.constant(np.random.random((rnn_size, 1)), dtype="float32")
W_inputs_tiled = tf.tile(tf.expand_dims(W_inputs, 0), [batch_size, 1, 1])
#W_pointerRNN_tiled = tf.tile(tf.expand_dims(W_pointerRNN, 0), [batch_size, 1, 1])
v_tiled = tf.tile(tf.expand_dims(v, 0), [batch_size, 1, 1])

In [ ]:
inputs = tf.constant(np.random.random((batch_size, max_seq_length, rnn_size)), dtype="float32")
def compute_lengths(inputs):
    used = tf.sign(tf.reduce_max(tf.abs(inputs), reduction_indices=2))
    lengths = tf.reduce_sum(used, reduction_indices=1)
    lengths = tf.cast(lengths, tf.int32) #lengths must be integers
    return lengths
seq_lengths = compute_lengths(inputs)

In [ ]:
#TODO: use attention-pooling over the question representation for the initialize hidden vector of pointer network
def loop_fn(time, prev_output, prev_state, prev_loop_state):
    print('start loop', time)
    next_output = prev_output
    if prev_output is None:
        next_state = cell.zero_state(batch_size, tf.float32)
    else:
        next_state = prev_state
    next_loop_state = None
    def get_next_input():
        print("computing logits")
        print("prev_output", prev_output)
        if prev_output is None:
            logits = tf.matmul(tf.tanh(tf.matmul(inputs, W_inputs_tiled)), v_tiled) #batch_size x seq_length x rnn_size
        else:
            print("i have prev output now")
            logits = tf.matmul(
                             tf.tanh(tf.matmul(inputs, W_inputs_tiled) + 
                                     tf.tile(tf.expand_dims(tf.matmul(prev_output, W_pointerRNN), 1), [1, max_seq_length, 1]))
                             , v_tiled) #batch_size x seq_length x rnn_size
        print("computing probs")
        predicted_probs = tf.nn.softmax(logits, dim=1) #a
        print("predicted_probs")
        weighted_input = tf.reduce_sum(tf.multiply(predicted_probs, inputs), axis=1) #c #batch_size x rnn_size
        print("returning weighted_input")
        return weighted_input #weighted input is next input
    print("1")
    elements_finished = (time >= seq_lengths) #this operation produces boolean tensor of [batch_size] defining if corresponding sequence has ended
    print("2")
    finished = tf.reduce_all(elements_finished) #AND operation over all batches. True if all batches finished.
    time += 1
    print("3")
    next_input = tf.cond(finished, lambda: tf.zeros([batch_size, rnn_size], dtype=tf.float32), get_next_input)
    print("4")
    return elements_finished, next_input, next_state, next_output, next_loop_state

In [ ]:
with tf.variable_scope(tf.get_variable_scope(), reuse=False):
    cell = tf.contrib.rnn.LSTMCell(rnn_size)
    outputs_ta, final_state, _ = tf.nn.raw_rnn(cell, loop_fn)
    outputs = outputs_ta.stack()
    #predicted_idx = tf.argmax(predicted_probs) #p

In [ ]:
tf.shape(outputs)

In [ ]:
outputs.get_shape().as_list()

In [ ]:
inputs = tf.constant(np.random.random((batch_size, max_seq_length, rnn_size)), dtype="float32")
inputs_ta = tf.TensorArray(dtype=tf.float32, size=max_seq_length)
inputs_ta = inputs_ta.unstack(inputs)

In [ ]:
inputs_ta.size().eval()

In [ ]:
inputs_ta.read(3)

In [ ]:
inputs = tf.constant(np.random.random((max_seq_length, batch_size, rnn_size)), dtype="float32")
inputs_ta = tf.TensorArray(dtype=tf.float32, size=max_seq_length)
inputs_ta = inputs_ta.unstack(inputs)

In [ ]:
inputs_ta.read(3)

In [ ]:
cell = tf.contrib.rnn.LSTMCell(rnn_size, reuse=True)
outputs_ta, final_state, _ = tf.nn.raw_rnn(cell, loop_fn)
outputs = outputs_ta.stack()
outputs_ta, final_state, _ = tf.nn.raw_rnn(cell, loop_fn)
outputs = outputs_ta.stack()
predicted_idx = tf.argmax(self.predicted_probs) #p

In [ ]:
#TODO: use attention-pooling over the question representation for the initialize hidden vector of pointer network
def loop_fn(time, prev_output, prev_state, prev_loop_state):
    print('start loop')
    next_output = prev_output
    if prev_output is None:
        next_cell_state = cell.zero_state(batch_size, tf.float32)
    else:
        next_cell_state = prev_state
    next_loop_state = None
    def get_next_input():
        print("computing logits")
        print(inputs_ta.read(time))
        print(prev_output)
        if prev_output is None:
            logits = tf.matmul(v, tf.tanh(tf.matmul(inputs_ta.read(time), W_inputs)))
        else:
            logits = tf.matmul(tf.tanh(tf.matmul(inputs_ta.read(time), W_inputs) + tf.matmul(prev_output, W_pointerRNN)), v) #s #at each step network has access to all paragraph inputs, plus the previous RNN outputs
        print("computing probs")
        predicted_probs = tf.nn.softmax(logits) #a
        print("computing weighted_input")
        weighted_input = tf.reduce_sum(tf.multiply(predicted_probs, inputs)) #c
        return weighted_input #weighted input is next input
    print("1")
    elements_finished = (time >= seq_lengths) #this operation produces boolean tensor of [batch_size] defining if corresponding sequence has ended
    print("2")
    finished = tf.reduce_all(elements_finished) #AND operation over all batches. True if all batches finished.
    time += 1
    print("3")
    next_input = tf.cond(finished, lambda: tf.zeros([batch_size, rnn_size], dtype=tf.float32), get_next_input)
    print("4")
    return elements_finished, next_input, next_state, next_output, next_loop_state

In [ ]:
inputs = tf.constant(np.random.random((batch_size, max_seq_length, rnn_size)))
def compute_lengths(inputs):
    used = tf.sign(tf.reduce_max(tf.abs(inputs), reduction_indices=2))
    lengths = tf.reduce_sum(used, reduction_indices=1)
    lengths = tf.cast(lengths, tf.int32) #lengths must be integers
    return lengths
seq_lengths = compute_lengths(inputs)
seq_lengths.eval()
def loop_fn(time, cell_output, cell_state, loop_state):
    print("start loop")
    emit_output = cell_output  # == None for time == 0
    if cell_output is None:  # time == 0
        next_cell_state = cell.zero_state(batch_size, tf.float32)
    else:
        next_cell_state = cell_state
    print("1")
    elements_finished = (time >= seq_lengths)
    print("2")
    finished = tf.reduce_all(elements_finished)
    print("3")
    next_input = tf.cond(
        finished,
        lambda: tf.zeros([batch_size, rnn_size], dtype=tf.float32),
        lambda: inputs_ta.read(time))
    print("4")
    next_loop_state = None
    print("5")
    print("elements_finished", elements_finished)
    print("next_input", next_input)
    print("next_cell_state", next_cell_state)
    print("next_loop_state", next_loop_state)
    return elements_finished, next_input, next_cell_state, emit_output, next_loop_state

max_seq_length = 100
batch_size = 10
rnn_size = 50
W_inputs = tf.constant(np.random.random((rnn_size, rnn_size)))
W_pointerRNN = tf.constant(np.random.random((rnn_size, rnn_size)))
v = tf.constant(np.random.random((1, rnn_size)))
inputs = tf.constant(np.random.random((max_seq_length, batch_size, rnn_size)))
inputs_ta = tf.TensorArray(dtype=tf.float32, size=max_seq_length)
inputs_ta = inputs_ta.unstack(inputs)
cell = tf.contrib.rnn.LSTMCell(rnn_size, reuse=True)
outputs_ta, final_state, _ = tf.nn.raw_rnn(cell, loop_fn)
outputs = outputs_ta.stack()